## Output Parsing

Language models output text. But there are times where you want to get more structured information than just text back

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

- **Get format instructions**: A method which returns a string containing instructions for how the output of a language model should be formatted.
- **Parse**: A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

- Output Parsing
    - StrOutputParser
    - JsonOutputParser
    - CSV Output Parser
    - Datatime Output Parser
    - Structured Output Parser (Pydanitc or Json)


### The .with_structured_output() method
- This method takes a schema as input which specifies the names, types, and descriptions of the desired output attributes.
-  The schema can be specified as a TypedDict class, JSON Schema or a Pydantic class.


In [1]:
from langchain_ollama import ChatOllama

from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate
                                        )

from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434"
model = 'llama3.2:3b'

llm = ChatOllama(base_url=base_url, model=model)

llm.invoke("Tell me a joke about cats")

In [ ]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

In [ ]:
structured_llm

### `Pydantinc` Output Parser

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

parser = PydanticOutputParser(pydantic_object=Joke)

# print(parser.get_format_instructions())

prompt = PromptTemplate(
    template='''Answer the user query.
                {format_instruction}
                {query}''',
    input_variables=['query'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)


chain = prompt | llm 
output = chain.invoke({'query':"Tell me a joke about cats"})
print(output.content)

parser.invoke(output)

In [ ]:
chain = prompt | llm | parser
output = chain.invoke({'query':"Tell me a joke about cats"})
output

### `JSON` Output Parser

- Output parsers accept a string or BaseMessage as input and can return an arbitrary type.



In [ ]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(pydantic_object=Joke)
# print(parser.get_format_instructions())

prompt = PromptTemplate(
    template='''Answer the user query. You should answer only as per provided format.
                {format_instruction}
                {query}''',
    input_variables=['query'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)


chain = prompt | llm 
output = chain.invoke({'query':"Tell me a joke about cats"})
print(output.content)


chain = prompt | llm | parser
output = chain.invoke({'query':"Tell me a joke about cats"})
output

### CSV Output Parser

- This output parser can be used when you want to return a list of comma-separated items.



In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | llm | output_parser

output = chain.invoke({"subject": "generate my website seo keywords. I have content about langchain. Do not write preambles or explanation."})
print(output)

### Datatime Output Parser

- Gives output in datetime format. Sometimes throws error if the LLM output is not in datetime format.

In [ ]:
from langchain.output_parsers import DatetimeOutputParser


output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt | llm | output_parser

# output = chain.invoke({"question": "What is the current time?"})
output = chain.invoke({"question": "When the America got discovered?"})

output